In [ ]:
import pandas as pd
import pandas as pd
import re
import openpyxl
import datetime as dt
import sqlite3
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import decimal

import os
import sys
# SRC_DIR = r'/home/pp/program/jupyter/'
sys.path.append(os.path.dirname(r'/home/pp/program/jupyter/'))
from config import *

# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet(TEPLOTY_SK_DIR + 'teploty_sk.parquet')
tf =df.copy()

print(df.head())

In [ ]:
dt = tf[['Cas_CET', 'Teplota']]
print(dt.head())
dt.info()
dt['Teplota'].astype(float).describe()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
sns.boxplot(x=dt['Teplota'].astype(float))
plt.show()

In [ ]:
sns.swarmplot(x=dt['Teplota'].astype(float).head(1000))
plt.show()

In [ ]:
dta=df.loc[['Dudince', 'Žilina','Chopok'],['Teplota','Cas_CET']]
dta=df.loc[['Dudince', 'Chopok'],['Teplota','Cas_CET']]

plt.figure(figsize=(25, 10))
data=dta.loc[['Dudince'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='o', linestyle='-')
data=dta.loc[['Chopok'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='o', linestyle='-')
plt.xlabel('Cas_CET')
plt.ylabel('Teplota')
plt.title('Teplota over Time')
plt.xticks(rotation=90)
plt.tight_layout()
plt.size=(150, 10)
plt.show()

In [ ]:
# dta=df.loc[['Dudince', 'Žilina','Chopok'],['Teplota','Cas_CET']]
# dta=df.loc[['Dudince', 'Chopok'],['Teplota','Cas_CET']]

plt.figure(figsize=(25, 10))
data=df.loc[['Žilina'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='', linestyle='-')
data=df.loc[['Chopok'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='', linestyle='-')
plt.xlabel('Cas_CET')
plt.ylabel('Teplota')
plt.title('Teplota over Time')
plt.xticks(rotation=90)
plt.tight_layout()
plt.size=(150, 10)
plt.show()

In [ ]:
# Resample to daily frequency, calculating the mean temperature for each day
dmi = df.set_index(['Cas_CET']).groupby('Stanica').resample('1D')['Teplota'].mean()
# Calculate a rolling mean with a window of 5 days
dmroll = dmi.rolling(window=5).mean()
# Reset index to turn the MultiIndex into columns
dmroll = dmroll.reset_index().set_index(['Stanica'])
dmi = dmi.reset_index().set_index(['Stanica'])

# For multiple aggregations
dfMultiple = df.set_index(['Cas_CET']).groupby('Stanica').resample('1D').agg({'Teplota': ['mean', 'max', 'min']})
# Reset index to turn the MultiIndex into columns
dfMultiple = dfMultiple.reset_index().set_index(['Stanica'])
print(dfMultiple.head(10))


In [ ]:




# df = dmi.set_index('Stanica')
plt.figure(figsize=(25, 10))
data=df.loc[['Žilina - letisko'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='', linestyle='-')
data=dmi.loc[['Žilina - letisko'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'], data['Teplota'],marker='', color='#FF0000', linestyle='-')
data=dmroll.loc[['Žilina - letisko'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET']  - pd.DateOffset(days=0), data['Teplota'],marker='', color='#00FF00', linestyle='-')


data=df.loc[['Chopok'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET'] , data['Teplota'],marker='', color="#2F00FF", linestyle='-')

data=dmroll.loc[['Chopok'],:].sort_values(by=['Cas_CET'])
plt.plot(data['Cas_CET']  - pd.DateOffset(days=2), data['Teplota'],marker='', color='#00FF00', linestyle='-')

plt.xlabel('Cas_CET')
plt.ylabel('Teplota')
plt.title('Teplota over Time')
plt.xticks(rotation=90)
plt.tight_layout()
plt.size=(150, 10)
plt.show()



In [ ]:
dmi = df.set_index(['Cas_CET']).groupby('Stanica').resample('1ME')['Teplota'].mean()
dmi.info()
dmi=dmi.reset_index()
dmi=dmi.set_index('Stanica')
dmi.info()
df.index
dmi.index
dmi.loc[['Žilina - letisko']]


In [ ]:
brezno = df[df['Stanica'] == 'Brezno']
dfbr = brezno.set_index(['Cas_CET']).resample('D').Teplota.agg(['min', 'max', 'mean', 'median' ])
dfbr.reset_index(inplace=True)
dfbr['Cas_CET'] = dfbr['Cas_CET'].dt.strftime('%Y-%m-%d')
dfbr.info()
dfbr


In [ ]:
br = brezno[brezno['Cas_CET'] < '2025-08-01']
br = br.loc[:,['Cas_CET','Teplota']]
br['Datum'] = br['Cas_CET'].dt.strftime('%Y-%m-%d')
# br['Cas_CET'].strftime('%Y-%m-d%')
# seria = br['Cas_CET'].copy()
# seria.info()
# br['Datum'] = seria.dt.strftime('%Y-%m-%d')
br
